### Choose model

In [8]:
#modelName = 'ResNet50'
#modelName = 'Vgg16'
#modelName = 'InceptionV3'
#modelName = 'InceptionResNetV2'
modelName = 'Xception'

In [1]:
!ls '/kaggle/input/inaturalist/inaturalist_12K'

train_val  val


In [3]:
!ls '/kaggle/input/xception/'

xception.py


In [4]:
import torch
import torch.nn as nn
from torchvision.transforms import RandomResizedCrop, RandomHorizontalFlip, Resize, CenterCrop, ToTensor, Normalize, Compose
import torchvision
import torch.nn.functional as F
import torch.optim as optim
import matplotlib.pyplot as plt
import numpy as np
import shutil
import sys
sys.path.append('/kaggle/input/xception/')
from xception import*
from inceptionresnetv2 import*

## Dataset info
iNaturalist = {
    'Normalize': {
        'mean': (0.485, 0.456, 0.406),
        'std':  (0.229, 0.224, 0.225)
    }
}

## Dataloaders
def data_loader(train_data, val_data, test_data, batchSize):
    train_dataLoader = torch.utils.data.DataLoader(train_data, batch_size=batchSize, shuffle=True)
    val_dataLoader = torch.utils.data.DataLoader(val_data, batch_size=batchSize, shuffle=True)
    test_dataLoader = torch.utils.data.DataLoader(test_data, batch_size=batchSize, shuffle=False)
    loaders = {
        'train' : train_dataLoader,
        'valid' : val_dataLoader,
        'test'  : test_dataLoader
    }
    return loaders

## getModdel fn
def getModel(modelName):
    import torchvision as tv 
    model = None
    
    if modelName == 'InceptionV3':
        model = tv.models.inception_v3(pretrained=True)
    elif modelName == 'InceptionResNetV2':
        model = inceptionresnetv2(pretrained='imagenet')
    elif modelName == 'ResNet50':
        model = tv.models.resnet50(pretrained=True)
    elif modelName == 'Xception':
        model = xception(pretrained=True)
    elif modelName == 'Vgg16':
        model = tv.models.vgg16(pretrained=True)
        
    return model

## transforms to match model input dims
def transform(modelName):
    if modelName == 'Xception' or modelName == 'InceptionV3' or modelName == 'InceptionResNetV2':
        resize = 299
        val_resize = 333
        val_center_crop = resize
        
    
    else:
        resize = 224
        val_resize = 256
        val_center_crop = resize
    
    
    train_t = Compose([RandomResizedCrop(resize),
                       RandomHorizontalFlip(),
                       ToTensor(),
                       Normalize(**iNaturalist['Normalize'])])
    valid_t = Compose([Resize(val_resize),
                       CenterCrop(resize),
                       ToTensor(),
                       Normalize(**iNaturalist['Normalize'])])
    test_t = Compose([Resize((resize,resize)), 
                      ToTensor(), 
                      Normalize(**iNaturalist['Normalize'])])
    
    transforms = {
        'training':   train_t,
        'validation': valid_t,
        'test': test_t
    }
    
    return transforms

## Load dataset fn
def load_datasets(modelName):
    transforms=transform(modelName)
    trainset  = torchvision.datasets.ImageFolder('/kaggle/input/inaturalist/inaturalist_12K/train_val/train', transforms['training'])
    valset    = torchvision.datasets.ImageFolder('/kaggle/input/inaturalist/inaturalist_12K/train_val/val', transforms['validation'])
    testset   = torchvision.datasets.ImageFolder('/kaggle/input/inaturalist/inaturalist_12K/val', transforms['test'])
    
    return trainset, valset, testset

## Save chkpt
def save_ckp(state, is_best, checkpoint_path, best_model_path):
    """
    state: checkpoint we want to save
    is_best: is this the best checkpoint; min validation loss
    checkpoint_path: path to save checkpoint
    best_model_path: path to save best model
    """
    f_path = checkpoint_path
    # save checkpoint data to the path given, checkpoint_path
    torch.save(state, f_path)
    # if it is a best model, min validation loss
    if is_best:
        best_fpath = best_model_path
        # copy that checkpoint file to best path given, best_model_path
        shutil.copyfile(f_path, best_fpath)


## load chkpt
def load_ckp(checkpoint_fpath, model, optimizer):
    """
    checkpoint_path: path to save checkpoint
    model: model that we want to load checkpoint parameters into       
    optimizer: optimizer we defined in previous training
    """
    # load check point
    checkpoint = torch.load(checkpoint_fpath)
    # initialize state_dict from checkpoint to model
    model.load_state_dict(checkpoint['state_dict'])
    # initialize optimizer from checkpoint to optimizer
    optimizer.load_state_dict(checkpoint['optimizer'])
    # initialize valid_loss from checkpoint to valid_loss
    valid_loss = checkpoint['valid_loss']
    # initialize valid_acc from checkpoint to valid_acc
    valid_acc = checkpoint['valid_acc']
    # initialize valid_loss from checkpoint to valid_loss
    valid_loss_min = checkpoint['valid_loss_min']
    # return model, optimizer, epoch value, min validation loss 
    return model, optimizer, checkpoint['epoch'], valid_loss.item(), valid_acc, valid_loss_min

In [5]:
import os
!pip install wandb
!pip install wandb --upgrade
import wandb

     |████████████████████████████████| 2.1 MB 865 kB/s eta 0:00:01
  Attempting uninstall: wandb
    Found existing installation: wandb 0.10.23
    Uninstalling wandb-0.10.23:
      Successfully uninstalled wandb-0.10.23


In [6]:
wandb.login()

wandb: You can find your API key in your browser here: https://wandb.ai/authorize


wandb: Paste an API key from your profile and hit enter:  ········································


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [7]:
# check if CUDA is available
use_cuda = torch.cuda.is_available()
if use_cuda == True:
    device = torch.device("cuda")
else:
    device = torch.device("cpu")

In [9]:
wandb.init(project=modelName+'_Sweep_1')

In [9]:
try:
    os.makedirs(modelName+"/checkpoint")
    os.makedirs(modelName+"/best_model")
except:
    print("directory already present")

In [10]:
ckp_path = "./"+modelName+"/checkpoint/current_checkpoint.pt"
best_ckp_path = "./"+modelName+"/best_model/best_model.pt"

In [11]:
if modelName =='Xception':
    batch_size = 4
else: 
    batch_size = 8

In [12]:
def model_change_classifier(model):
    #modelName = model.name
    if modelName == 'InceptionV3':
        model.fc = nn.Sequential(nn.Linear(model.fc.in_features,500),
                                 nn.ReLU(),
                                 nn.Dropout(),
                                 nn.Linear(500,10))


    if modelName == 'Vgg16':
        model.classifier[6] = nn.Sequential(nn.Linear(model.classifier[6].in_features,500),
                                 nn.ReLU(),
                                 nn.Dropout(),
                                 nn.Linear(500,10))


    if modelName == 'ResNet50':
        model.fc = nn.Sequential(nn.Linear(model.fc.in_features,500),
                             nn.ReLU(),
                             nn.Dropout(),
                             nn.Linear(500,10))

    if modelName == 'Xception':
        model.fc = nn.Sequential(nn.Linear(model.fc.in_features,500),
                             nn.ReLU(),
                             nn.Dropout(),
                             nn.Linear(500,10))
    return

In [13]:
def train(start_epochs, n_epochs, valid_loss_min_input, loaders, model, optimizer, criterion, use_cuda, checkpoint_path, best_model_path):
    
    valid_loss_min = valid_loss_min_input 
    
    for epoch in range(start_epochs, start_epochs+n_epochs):
        train_loss = 0.0
        valid_loss = 0.0
        
        ###################
        # train the model #
        ###################
        model.train()
        tnum_correct = 0
        tnum_examples = 0
        for batch_idx, (data, target) in enumerate(loaders['train']):
            # move to GPU
            if use_cuda:
                data, target = data.cuda(), target.cuda()
                
            optimizer.zero_grad()
            if model.name == 'InceptionV3':
                output, aux_output = model(data)
                loss1 = criterion(output, target)
                loss2 = criterion(aux_output, target)
                loss = loss1 + 0.4*loss2
                
            else:
                output = model(data)
                loss = criterion(output, target)
            
            correct = torch.eq(torch.max(F.softmax(output, dim=1), dim=1)[1],target).view(-1)
            tnum_correct += torch.sum(correct).item()
            tnum_examples += correct.shape[0]
            loss.backward()
            optimizer.step()
            train_loss = train_loss + ((1 / (batch_idx + 1)) * (loss.data - train_loss))
        train_acc = tnum_correct / tnum_examples
        
        ######################    
        # validate the model #
        ######################
        model.eval()
        num_correct = 0
        num_examples = 0
        for batch_idx, (data, target) in enumerate(loaders['valid']):
            
            if use_cuda:
                data, target = data.cuda(), target.cuda()
            
            output = model(data)
            loss = criterion(output, target)
            valid_loss = valid_loss + ((1 / (batch_idx + 1)) * (loss.data - valid_loss))
            
            correct = torch.eq(torch.max(F.softmax(output, dim=1), dim=1)[1],target).view(-1)
            num_correct += torch.sum(correct).item()
            num_examples += correct.shape[0]
            
        train_loss = train_loss/len(loaders['train'].dataset)
        valid_loss = valid_loss/len(loaders['valid'].dataset)
        valid_acc = num_correct / num_examples
        
        print('Epoch: {} \tTraining Loss: {:.6f} \tTrain Accuracy: {:.2f} \tValidation Loss: {:.6f} \tvalidation Accuracy: {:.2f}'.format(
            epoch, 
            train_loss,
            train_acc,
            valid_loss,
            valid_acc
            ))
        
        wandb.log({'epoch': epoch,'train loss': train_loss,'train accuracy': train_acc,
                   'val loss': valid_loss, 'val accuracy': valid_acc})
        
        checkpoint = {
            'epoch': epoch + 1,
            'valid_loss': valid_loss,
            'valid_acc': valid_acc,
            'valid_loss_min': valid_loss_min,
            'state_dict': model.state_dict(),
            'optimizer': optimizer.state_dict(),
        }
        
        save_ckp(checkpoint, False, checkpoint_path, best_model_path)
        
        if valid_loss <= valid_loss_min:
            print('Validation loss decreased ({:.6f} --> {:.6f}).  Saving model ...'.format(valid_loss_min,valid_loss))
            # save checkpoint as best model
            save_ckp(checkpoint, True, checkpoint_path, best_model_path)
            valid_loss_min = valid_loss
            
    # return trained model
    return model

In [14]:
def sp_train():
    config_defaults = {
        'model_name':modelName,
        'preTrain_epochs': 3,
        'fineTune_epochs': 5,
        'learning_rate_1': 1e-4,
        'learning_rate_2': 1e-4,
        'batchnorm_pretrain':'YES',
        'optimizer': 'sgd',
        'batch_size': batch_size
    }
    # Initialize a new wandb run
    wandb.init(config=config_defaults)
    # Config is a variable that holds and saves hyperparameters and inputs
    config = wandb.config
    run_name="pT_ep:["+str(config.preTrain_epochs)+"] fT_ep:["+str(config.fineTune_epochs)+"] lr1:["+str(config.learning_rate_1)+"] lr2:["+str(config.learning_rate_2)+"] op:["+config.optimizer+"] bs:["+str(config.batch_size)+"] BNpT:["+config.batchnorm_pretrain+"]"
    wandb.run.name=run_name
    #wandb.init(config=config_defaults, name= run_name)
    
    model = getModel(modelName)
    model.name = modelName
    
    
    datasetTrain, datasetVal, datasetTest = load_datasets(modelName)
    
    loaders = data_loader(datasetTrain, datasetVal, datasetTest, config.batch_size)
    
    # Get Batchnorm Layers
    msBN = list(filter(lambda m: type(m) == torch.nn.modules.BatchNorm2d, model.modules()))
    
    # Freezing layer
    for param in model.parameters():
        if param.requires_grad:
            param.requires_grad=False
    
    model_change_classifier(model)
    
    # Batchnorm layers unfreeze
    if config.batchnorm_pretrain=='YES':
        for i, m in enumerate(msBN):
            for param in m.parameters():
                param.requires_grad= True
                
    model = model.to(device)
    
    if config.optimizer=='sgd':
        optimizer = optim.SGD(model.parameters(), lr=config.learning_rate_1, momentum = 0.9)
    elif config.optimizer=='adam':
        optimizer = optim.Adam(model.parameters(), lr=config.learning_rate_1, betas=(0.9, 0.999))
    criterion = nn.CrossEntropyLoss()
    
    trained_model = train(start_epochs = 1,
                      n_epochs = config.preTrain_epochs,
                      valid_loss_min_input = np.Inf,
                      loaders = loaders,
                      model = model,
                      optimizer = optimizer,
                      criterion = criterion,
                      use_cuda = use_cuda,
                      checkpoint_path = ckp_path,
                      best_model_path = best_ckp_path
                     )
    
    model, optimizer, start_epoch, valid_loss, valid_acc, valid_loss_min = load_ckp(ckp_path, model, optimizer)
    
    for name, param in model.named_parameters():
        param.requires_grad = True
        
    for g in optimizer.param_groups:
        g['lr'] = config.learning_rate_2
        
    trained_model = train(start_epochs = start_epoch,
                      n_epochs = config.fineTune_epochs,
                      valid_loss_min_input = valid_loss_min,
                      loaders = loaders,
                      model = model,
                      optimizer = optimizer,
                      criterion = criterion,
                      use_cuda = use_cuda,
                      checkpoint_path = ckp_path,
                      best_model_path = best_ckp_path
                     )

In [15]:
sweep_config = {
    'method': 'grid', 
    'metric': {
      'name': 'val accuracy',
      'goal': 'maximize'   
    },
    'parameters': {
        'Model Name': {
            'values':[modelName]
        },
        'preTrain_epochs': {
            'values':[3]
        },
        'fineTune_epochs': {
            'values': [5]
        },
        'learning_rate_1': {
            'values':[0.0001, 0.001] 
        },
        'learning_rate_2':{
            'values':[0.0001, 0.001]
        },
        'batchnorm_pretrain':{
            'values': ['YES', 'NO']
        },
        'optimizer': {
            'values':['adam','sgd']
        },
        'batch_size': {
            'values':[batch_size]
        }
    }
}

In [20]:
sweep_id = wandb.sweep(sweep_config, project=modelName+"_Sweep_1")

Create sweep with ID: mikg9z8b
Sweep URL: https://wandb.ai/rayanz/Xception_Sweep_1/sweeps/mikg9z8b


In [ ]:
wandb.agent(sweep_id, sp_train)

wandb: Sweep Agent: Waiting for job.
wandb: Job received.
wandb: Agent Starting Run: wwsujkya with config:
wandb: 	Model Name: Xception
wandb: 	batch_size: 4
wandb: 	batchnorm_pretrain: YES
wandb: 	fineTune_epochs: 5
wandb: 	learning_rate_1: 0.0001
wandb: 	learning_rate_2: 0.0001
wandb: 	optimizer: adam
wandb: 	preTrain_epochs: 3


Epoch: 1 	Training Loss: 0.000177 	Train Accuracy: 0.57 	Validation Loss: 0.000319 	validation Accuracy: 0.82
Validation loss decreased (inf --> 0.000319).  Saving model ...
Epoch: 2 	Training Loss: 0.000125 	Train Accuracy: 0.68 	Validation Loss: 0.000276 	validation Accuracy: 0.83
Validation loss decreased (0.000319 --> 0.000276).  Saving model ...
Epoch: 3 	Training Loss: 0.000113 	Train Accuracy: 0.71 	Validation Loss: 0.000260 	validation Accuracy: 0.83
Validation loss decreased (0.000276 --> 0.000260).  Saving model ...
Epoch: 4 	Training Loss: 0.000118 	Train Accuracy: 0.70 	Validation Loss: 0.000279 	validation Accuracy: 0.83
Epoch: 5 	Training Loss: 0.000100 	Train Accuracy: 0.74 	Validation Loss: 0.000299 	validation Accuracy: 0.83
Epoch: 6 	Training Loss: 0.000093 	Train Accuracy: 0.76 	Validation Loss: 0.000292 	validation Accuracy: 0.83
Epoch: 7 	Training Loss: 0.000085 	Train Accuracy: 0.78 	Validation Loss: 0.000315 	validation Accuracy: 0.83
Epoch: 8 	Training Loss: 0.0

epoch,8
train loss,8e-05
train accuracy,0.79035
val loss,0.00035
val accuracy,0.8215
_runtime,2728
_timestamp,1618036532
_step,7


epoch,▁▂▃▄▅▆▇█
train loss,█▄▃▄▂▂▁▁
train accuracy,▁▅▆▅▆▇██
val loss,▅▂▁▂▄▃▅█
val accuracy,▁▅█▅▇▆▅▃
_runtime,▁▂▃▄▅▆▇█
_timestamp,▁▂▃▄▅▆▇█
_step,▁▂▃▄▅▆▇█


wandb: Agent Starting Run: wa2i4dit with config:
wandb: 	Model Name: Xception
wandb: 	batch_size: 4
wandb: 	batchnorm_pretrain: YES
wandb: 	fineTune_epochs: 5
wandb: 	learning_rate_1: 0.0001
wandb: 	learning_rate_2: 0.0001
wandb: 	optimizer: sgd
wandb: 	preTrain_epochs: 3


...

Epoch: 1 	Training Loss: 0.000193 	Train Accuracy: 0.54 	Validation Loss: 0.000398 	validation Accuracy: 0.79
Validation loss decreased (inf --> 0.000398).  Saving model ...
Epoch: 2 	Training Loss: 0.000143 	Train Accuracy: 0.65 	Validation Loss: 0.000333 	validation Accuracy: 0.81
Validation loss decreased (0.000398 --> 0.000333).  Saving model ...
Epoch: 3 	Training Loss: 0.000134 	Train Accuracy: 0.66 	Validation Loss: 0.000308 	validation Accuracy: 0.81
Validation loss decreased (0.000333 --> 0.000308).  Saving model ...
Epoch: 4 	Training Loss: 0.000285 	Train Accuracy: 0.15 	Validation Loss: 0.001165 	validation Accuracy: 0.16
Epoch: 5 	Training Loss: 0.000282 	Train Accuracy: 0.16 	Validation Loss: 0.001387 	validation Accuracy: 0.19
Epoch: 6 	Training Loss: 0.000279 	Train Accuracy: 0.17 	Validation Loss: 0.001066 	validation Accuracy: 0.24
Epoch: 7 	Training Loss: 0.000271 	Train Accuracy: 0.20 	Validation Loss: 0.001043 	validation Accuracy: 0.26
Epoch: 8 	Training Loss: 0.0

epoch,8
train loss,0.00026
train accuracy,0.22353
val loss,0.00098
val accuracy,0.281
_runtime,2482
_timestamp,1618060499
_step,7


epoch,▁▂▃▄▅▆▇█
train loss,▄▁▁███▇▇
train accuracy,▆██▁▁▁▂▂
val loss,▂▁▁▇█▆▆▅
val accuracy,███▁▁▂▂▂
_runtime,▁▂▂▄▅▆▇█
_timestamp,▁▂▂▄▅▆▇█
_step,▁▂▃▄▅▆▇█


wandb: Agent Starting Run: f6uboqz0 with config:
wandb: 	Model Name: Xception
wandb: 	batch_size: 4
wandb: 	batchnorm_pretrain: NO
wandb: 	fineTune_epochs: 5
wandb: 	learning_rate_1: 0.0001
wandb: 	learning_rate_2: 0.001
wandb: 	optimizer: sgd
wandb: 	preTrain_epochs: 3


Epoch: 1 	Training Loss: 0.000281 	Train Accuracy: 0.27 	Validation Loss: 0.001074 	validation Accuracy: 0.65
Validation loss decreased (inf --> 0.001074).  Saving model ...
Epoch: 2 	Training Loss: 0.000261 	Train Accuracy: 0.49 	Validation Loss: 0.000955 	validation Accuracy: 0.74
Validation loss decreased (0.001074 --> 0.000955).  Saving model ...
Epoch: 3 	Training Loss: 0.000234 	Train Accuracy: 0.56 	Validation Loss: 0.000794 	validation Accuracy: 0.77
Validation loss decreased (0.000955 --> 0.000794).  Saving model ...
Epoch: 4 	Training Loss: 0.000137 	Train Accuracy: 0.66 	Validation Loss: 0.000271 	validation Accuracy: 0.82
Validation loss decreased (0.000955 --> 0.000271).  Saving model ...
Epoch: 5 	Training Loss: 0.000104 	Train Accuracy: 0.74 	Validation Loss: 0.000248 	validation Accuracy: 0.84
Validation loss decreased (0.000271 --> 0.000248).  Saving model ...
Epoch: 6 	Training Loss: 0.000090 	Train Accuracy: 0.77 	Validation Loss: 0.000244 	validation Accuracy: 0.85


In [18]:
wandb.agent(sweep_id = 'mikg9z8b', project = modelName+'_Sweep_1', function = sp_train)

wandb: Sweep Agent: Waiting for job.
wandb: Job received.
wandb: Agent Starting Run: nx5ma5ly with config:
wandb: 	Model Name: Xception
wandb: 	batch_size: 4
wandb: 	batchnorm_pretrain: NO
wandb: 	fineTune_epochs: 5
wandb: 	learning_rate_1: 0.0001
wandb: 	learning_rate_2: 0.001
wandb: 	optimizer: sgd
wandb: 	preTrain_epochs: 3


Epoch: 1 	Training Loss: 0.000281 	Train Accuracy: 0.27 	Validation Loss: 0.001084 	validation Accuracy: 0.63
Validation loss decreased (inf --> 0.001084).  Saving model ...
Epoch: 2 	Training Loss: 0.000262 	Train Accuracy: 0.49 	Validation Loss: 0.000959 	validation Accuracy: 0.73
Validation loss decreased (0.001084 --> 0.000959).  Saving model ...
Epoch: 3 	Training Loss: 0.000235 	Train Accuracy: 0.56 	Validation Loss: 0.000805 	validation Accuracy: 0.75
Validation loss decreased (0.000959 --> 0.000805).  Saving model ...
Epoch: 4 	Training Loss: 0.000135 	Train Accuracy: 0.66 	Validation Loss: 0.000300 	validation Accuracy: 0.81
Validation loss decreased (0.000959 --> 0.000300).  Saving model ...
Epoch: 5 	Training Loss: 0.000103 	Train Accuracy: 0.74 	Validation Loss: 0.000253 	validation Accuracy: 0.85
Validation loss decreased (0.000300 --> 0.000253).  Saving model ...
Epoch: 6 	Training Loss: 0.000093 	Train Accuracy: 0.76 	Validation Loss: 0.000245 	validation Accuracy: 0.85


epoch,8
train loss,8e-05
train accuracy,0.78985
val loss,0.00024
val accuracy,0.856
_runtime,1922
_timestamp,1618078526
_step,7


epoch,▁▂▃▄▅▆▇█
train loss,█▇▆▃▂▁▁▁
train accuracy,▁▄▅▆▇███
val loss,█▇▆▂▁▁▁▁
val accuracy,▁▄▅▇██▇█
_runtime,▁▂▃▄▅▆▇█
_timestamp,▁▂▃▄▅▆▇█
_step,▁▂▃▄▅▆▇█


wandb: Sweep Agent: Waiting for job.
wandb: Sweep Agent: Exiting.
